In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [180]:
#KD树节点
class Node():
    def __init__(self,data=None,label=None,parent=None,left=None,right=None,ki=None,axis=None,is_call=0):
        self.data = data  #节点数据值
        self.label = label  #节点标签
        self.parent = parent  ##节点的父亲节点
        self.left = left  #节点的左子树
        self.right = right  #节点的右子树
        self.ki = ki  #切分轴
        self.axis = axis  #节点所处的划分空间轴
        self.is_call = is_call  #该节点是否被访问过 0表示没有被访问过，1被访问过(用于KNN算法)

In [89]:
#快速排序
def partition(seq,low,high):
    pi = seq[high]
    i = low-1
    for j in range(low,high):
        if(seq[j]<=pi):
            i+=1
            seq[i],seq[j] = seq[j],seq[i]
    seq[i+1],seq[high] = seq[high],seq[i+1]
    return i+1
        
def Quicksort(seq,low,high):
    if(low<high):
        t = partition(seq,low,high)
        Quicksort(seq,low,t-1)
        Quicksort(seq,t+1,high)


In [181]:
#KD树
class Kdtree():
    
    def __init__(self,sample,label):
        self.__root = self.__creat(sample,label)
    
    
    def __creat(self,sample,labels,parentNode=None):
        '''
            使用递归方式创建KD树
            sample：样本数据列表
            labels：样本标记列表
            parentNone：父亲节点
            return：返回根节点
        '''
        
        dataArray = np.array(sample)
        sampleNum,fea_list = dataArray.shape
        if sampleNum == 0:  ##若没有样本就直接返回
            return None
        ##使用各个特征值的方差判断出切分轴
        var_list = [np.var(dataArray[:,feature]) for feature in range(fea_list)]
        max_var_index = var_list.index(max(var_list))
        dataArray_index_sort = dataArray[:,max_var_index].argsort()
        mid_sample_index = dataArray_index_sort[sampleNum//2]
        if sampleNum == 1:  ##当剩一个样本时返回
            return Node(data=dataArray[mid_sample_index],label=labels[mid_sample_index],ki=max_var_index,axis=dataArray[mid_sample_index,max_var_index],parent=parentNode,is_call=0)
        node = Node(data=dataArray[mid_sample_index],label=labels[mid_sample_index],ki=max_var_index,axis=dataArray[mid_sample_index,max_var_index],is_call=0)
        if parentNode:
            node.parent = parentNode
        ##左子树
        left_sample = [dataArray[left_mid] for left_mid in dataArray_index_sort[:sampleNum//2]]
        left_labels = [labels[left_mid] for left_mid in dataArray_index_sort[:sampleNum//2]]
        left_child = self.__creat(sample=left_sample,labels=left_labels,parentNode=node)
        if sampleNum == 2:  ##如果是两个两样本就没有右子树
            right_child = None
        else:
            ##右子树
            right_sample = [dataArray[right_mid] for right_mid in dataArray_index_sort[sampleNum//2+1:]]
            right_labels = [labels[right_mid] for right_mid in dataArray_index_sort[sampleNum//2+1:]]
            right_child = self.__creat(sample=right_sample,labels=left_labels,parentNode=node)
        ##赋值节点的左右子树
        node.left = left_child
        node.right = right_child
        return node
    
    
    @property
    def root(self):
        return self.__root
    
    
    def __show_node(self,node):
        '''
        对kd树进行显示输出辅助函数
        '''
        
        if node == None:
            return None
        kd_dict = dict()
        kd_dict[tuple(node.data)] = {}
        kd_dict[tuple(node.data)]['label'] = node.label
        kd_dict[tuple(node.data)]['parent'] = tuple(node.parent.data) if node.parent else None
        kd_dict[tuple(node.data)]['ki'] = node.ki
        kd_dict[tuple(node.data)]['axis'] = node.axis
        kd_dict[tuple(node.data)]['left'] = self.__show_node(node.left)
        kd_dict[tuple(node.data)]['right'] = self.__show_node(node.right)
        return kd_dict
    
    
    def show_kd(self):
        '''
        对kd树进行显示
        '''
        return self.__show_node(node=self.__root)
    
    def parent_node(self,node):
        '''
        返回节点的父亲节点
        '''
        return node.parent

In [182]:
##查找叶子结点
def search_are(node,sample):
    '''
    从该输入节点开始比较输入数据向下查找到叶子结点
    node:开始查找的节点
    sample:需要预测的数据
    '''
    
    while(node):
        if sample[node.ki] <= node.data[node.ki]:
            if node.left == None:
                return node
            else:
                node = node.left
        else:
            if node.right == None:
                return node
            else:
                node = node.right
    return None



In [298]:
## KNN算法实现
def kd_knn(knode,sample,result,k):
    '''
    在构建好的kd树中查找离预测数据最近的k个点
    knode:kd树中开始查找的起始节点
    sample:需要预测的输入数据
    k:需要查找离预测数据最近点的个数
    result:结果数组，结果数组中存储的结果包括节点的数据，节点的标签，以及节点到预测节点的距离
    
    return:result结果数组
    '''
    
    '''
    过程一:从输入的起始点开始向下搜索得到叶子节点，并将该叶子节点标记为已访问，如此时结果数组中的不足k个结果就直接加入到结果数组中，
    若结果数组中的数据已满足k个就比较该叶子节点到预测点的距离与结果数组中最远的结果，若小与结果数组中最远的结果泽则替换该结果。
    '''
    
    node = search_are(knode,sample=sample)
    node.is_call = 1  ##将该节点标记为已访问
    distance = np.sqrt(np.sum(np.square(sample-node.data)))
    result_sample = list(node.data) + [node.label,distance]
    if result.shape[0] < k:  ##当结果列表中的样本数不足k时，直接加入
        result = np.insert(result,0,result_sample,0)
    else:
        max_distence_index = result[:,-1].argsort()[-1] 
        if result[max_distence_index][-1] > distance:
            result = np.delete(result,max_distence_index,0)
            result = np.insert(result,0,result_sample,0)
            
    '''
    过程二:若当前的节点为根节点则返回结果数据，若不是则向上爬一个节点（父亲节点），若向上爬的节点已被访问过则该已被访问的节点执行过程二，
    若没被访问将其标记为访问过，此时若结果数组的结果不满k个，就将该节点加入到结果数组中，若结果数组已满k个则比较该节点与预测节点的距离
    与结果数组中最远的结果，若小于最远的结果就替换最远的结果
    '''
    
    while(node.parent):
        node = node.parent
        if node.is_call == 0:
            node.is_call = 1
        else:
            continue
        distance = np.sqrt(np.sum(np.square(sample-node.data)))
        result_sample = list(node.data) + [node.label,distance]
        if result.shape[0] < k:
            result = np.insert(result,0,result_sample,0)
        else:
            max_distence_index = result[:,-1].argsort()[-1]  
            if result[max_distence_index][-1] > distance:
                result = np.delete(result,max_distence_index,0)
                result = np.insert(result,0,result_sample,0)
        '''
        计算预测节点到爬上来节点的切分轴，若小于结果数组中最远的结果，则该节点的其他分支可能存在小于结果数组中最远的结果，将该节点中
        未被访问的分支节点最为开始节点从过程一开始进行操作
        '''
        
        axis_distance = np.fabs(sample[node.ki]-node.axis)
        max_distence_index = result[:,-1].argsort()[-1]
        if axis_distance < result[max_distence_index][-1]:
            if node.right.is_call == 0:
                return kd_knn(node.right,sample,result,k)
            elif node.left.is_call == 0:
                return kd_knn(node.right,sample,result,k)
        return result

[[7.         2.         0.         2.23606798]
 [5.         4.         1.         1.        ]]


In [308]:
##测试函数
def knn_main():
    '''
    测试函数
    测试样本:forecast_sample = [3,4]
    样本空间:sample = [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]  label = [0,1,1,1,0,0]
    k=1
    '''
    
    sample = [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]  
    label = [0,1,1,1,0,0]
    kd = Kdtree(sample,label)
    result = np.array([[]])
    forecast_sample = [3,4]
    result.shape = (0,len(forecast_sample)+2)
    result = kd_knn(knode=kd.root,sample=forecast_sample,result=result,k=1)
    print(result)
    i = 0  ##结果数组中标记为0的个数
    j = 0  ##结果数据中标记为1的个数
    for re in result:
        if re[-2] == 0:
            i += 1
        elif re[-2] ==1:
            j += 1
    if i > j:
        print('预测结果为：0')
    elif j >= i:
        print('预测结果为：1')
knn_main()

[[2.         3.         0.         1.41421356]]
预测结果为：0
